In [3]:

import pandas as pd

archivo = 'dataset_2017_full_clean.csv'

# Cargar el dataset
df = pd.read_csv(archivo)

# Eliminar las filas que contengan twhashtag, twmention de la columna 'tweet'

df = df[~df['tweet'].str.contains('twhashtag|twmention', na=False)]

# Eliminar las comillas de las filas
df['tweet'] = df['tweet'].str.replace('"', '', regex=False)

# Eliminar los emojis de las filas
df['tweet'] = df['tweet'].str.replace(r'[^\x00-\x7F]+', '', regex=True)

# Eliminar lo que machee con esto @\S+ 
df['tweet'] = df['tweet'].str.replace(r'@\S+ ', '', regex=True)

# Guardar el dataset limpio
df.to_csv('comentarios_dataset_limpitov1.csv', index=False)


In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import re
import numpy as np
import json
from collections import Counter
from sklearn.model_selection import train_test_split

# Ruta al CSV
CSV_PATH = 'comentarios_dataset_limpitov1.csv'

# Parámetros
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100
TEST_SIZE = 0.2
RANDOM_STATE = 42

# Lista simple de stopwords en español
STOPWORDS = {
    'de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se',
    'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al',
    'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este',
    'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre',
    'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo',
    'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros',
    'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos',
    'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa',
    'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella',
    'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú',
    'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras',
    'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas',
    'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros',
    'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos',
    'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están',
    'esté', 'estés', 'estemos', 'estéis', 'estén', 'estaré', 'estarás',
    'estará', 'estaremos', 'estaréis', 'estarán', 'estaría', 'estarías',
    'estaríamos', 'estaríais', 'estarían', 'estaba', 'estabas', 'estábamos',
    'estabais', 'estaban', 'estuve', 'estuviste', 'estuvo', 'estuvimos',
    'estuvisteis', 'estuvieron', 'estuviera', 'estuvieras', 'estuviéramos',
    'estuvierais', 'estuvieran', 'estuviese', 'estuvieses', 'estuviésemos',
    'estuvieseis', 'estuviesen', 'estando', 'estado', 'estada', 'estados',
    'estadas', 'estad'
}

def limpiar_texto(texto):
    # 1) pasar a minúsculas
    texto = str(texto).lower()
    # 2) eliminar URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)
    # 3) eliminar caracteres que no sean letras, números o espacios
    texto = re.sub(r'[^a-záéíóúüñ0-9\s]', '', texto)
    # 4) normalizar espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    # 5) remover stopwords
    tokens = texto.split()
    tokens_filtrados = [tok for tok in tokens if tok not in STOPWORDS]
    return ' '.join(tokens_filtrados)

def construir_vocabulario(textos, max_size):
    contador = Counter()
    for t in textos:
        tokens = t.split()
        contador.update(tokens)
    mas_comunes = contador.most_common(max_size - 2)  # Reservar índices 0 y 1
    word2idx = {palabra: idx + 2 for idx, (palabra, _) in enumerate(mas_comunes)}
    word2idx['<OOV>'] = 1
    return word2idx

def texto_a_secuencia(texto, word2idx):
    return [word2idx.get(tok, 1) for tok in texto.split()]

def pad_sequence(seq, max_len):
    if len(seq) < max_len:
        return seq + [0] * (max_len - len(seq))
    else:
        return seq[:max_len]

def main():
    # 1) Cargar CSV
    df = pd.read_csv(CSV_PATH)
    
    # 2) Eliminar filas con sentiment = -1 (NONE)
    df = df[df['sentiment'] != -1]
    
    # 3) Limpiar textos y eliminar vacíos
    df['clean_tweet'] = df['tweet'].apply(limpiar_texto)
    df = df[df['clean_tweet'] != '']
    
    # 4) Codificar etiquetas
    #    0 = Neutral, 1 = Positivo, 2 = Negativo
    clases = sorted(df['sentiment'].unique())  # [0, 1, 2]
    mapeo_labels = {label: idx for idx, label in enumerate(clases)}
    df['label_idx'] = df['sentiment'].map(mapeo_labels)
    num_clases = len(clases)
    y = np.eye(num_clases)[df['label_idx']]
    
    # 5) Split train/test estratificado
    X = df['clean_tweet'].values
    y_labels = df['label_idx'].values
    X_train, X_test, y_train_idx, y_test_idx = train_test_split(
        X, y_labels,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        stratify=y_labels
    )
    y_train = np.eye(num_clases)[y_train_idx]
    y_test = np.eye(num_clases)[y_test_idx]
    
    # 6) Construir vocabulario solo con entrenamiento
    word2idx = construir_vocabulario(X_train, MAX_VOCAB_SIZE)
    
    # 7) Convertir textos a secuencias de índices
    seq_train = [texto_a_secuencia(t, word2idx) for t in X_train]
    seq_test = [texto_a_secuencia(t, word2idx) for t in X_test]
    
    # 8) Padding y truncamiento
    X_train_padded = np.array([pad_sequence(s, MAX_SEQUENCE_LENGTH) for s in seq_train], dtype=np.int32)
    X_test_padded = np.array([pad_sequence(s, MAX_SEQUENCE_LENGTH) for s in seq_test], dtype=np.int32)
    
    # 9) Guardar vectores y mapeos
    np.save('X_train_padded.npy', X_train_padded)
    np.save('X_test_padded.npy', X_test_padded)
    np.save('y_train.npy', y_train)
    np.save('y_test.npy', y_test)
    
    with open('word2idx.json', 'w', encoding='utf-8') as f:
        json.dump(word2idx, f, ensure_ascii=False, indent=2)
    mapeo_labels_str = {str(k): int(v) for k, v in mapeo_labels.items()}
    with open('label_mapping.json', 'w', encoding='utf-8') as f:
        json.dump(mapeo_labels_str, f, ensure_ascii=False, indent=2)
    
    print("Preprocesamiento completado.")
    print(f"  X_train_padded.npy → {X_train_padded.shape}")
    print(f"  X_test_padded.npy  → {X_test_padded.shape}")
    print(f"  y_train.npy        → {y_train.shape}")
    print(f"  y_test.npy         → {y_test.shape}")
    print("Mapas guardados: word2idx.json, label_mapping.json")

if __name__ == '__main__':
    main()


Preprocesamiento completado.
  X_train_padded.npy → (17577, 100)
  X_test_padded.npy  → (4395, 100)
  y_train.npy        → (17577, 3)
  y_test.npy         → (4395, 3)
Mapas guardados: word2idx.json, label_mapping.json
